In [ ]:
# Importando as bibliotecas necessárias
import pandas as pd
import os
import kagglehub
import shutil

In [ ]:
DATASETS = [
    "shivamb/amazon-prime-movies-and-tv-shows",
    "shivamb/disney-movies-and-tv-shows",
    "shivamb/netflix-shows",
]

current_directory = os.getcwd() 
DESTINATION_DIR = os.path.join(current_directory, "data") 

# 1. Cria a pasta 'data' se ela não existir
os.makedirs(DESTINATION_DIR, exist_ok=True)

def download_and_copy_dataset(dataset_name: str, destination_path: str):
    """Baixa um dataset, copia os arquivos para o destino e limpa o cache."""

    # 1. Baixa o dataset para o cache
    try:
        cache_path = kagglehub.dataset_download(dataset_name)
    except Exception as e:
        return

    # 2. Copia os arquivos do cache para o diretório de destino
    for item_name in os.listdir(cache_path):
        source = os.path.join(cache_path, item_name)
        destination = os.path.join(destination_path, item_name)

        # Copia apenas arquivos (ignorando subpastas)
        if os.path.isfile(source):
            shutil.copy2(source, destination) 

    # 3. Remove completamente a pasta do cache
    try:
        shutil.rmtree(cache_path)
    except OSError as e:
        print(f"  > AVISO: Não foi possível remover o cache: {e}")
        
for dataset in DATASETS:
    download_and_copy_dataset(dataset, DESTINATION_DIR)

print("\n\n--- Processo Finalizado ---")
print(f"Todos os arquivos dos datasets estão na pasta: {DESTINATION_DIR}")

In [ ]:
#Concatenar as base de dados 

DESTINATION_DIR = os.path.join(current_directory, "data") 

df_netflix = pd.read_csv(os.path.join(DESTINATION_DIR, 'netflix_titles.csv'))
df_disney = pd.read_csv(os.path.join(DESTINATION_DIR, 'disney_plus_titles.csv'))
df_amazon = pd.read_csv(os.path.join(DESTINATION_DIR, 'amazon_prime_titles.csv'))

df_netflix['streaming'] = 'Netflix'
df_disney['streaming'] = 'Disney+'
df_amazon['streaming'] = 'Prime Video'

dataframes_to_concat = [df_netflix, df_disney, df_amazon]

df_streaming = pd.concat(dataframes_to_concat, ignore_index=True)

df_streaming['date_added'] = df_streaming['date_added'].str.strip()

df_streaming['date_added'] = pd.to_datetime(df_streaming['date_added'], format='%B %d, %Y')

# Exibindo informações gerais para confirmar a junção e os tipos de dados
print("\nInformações do DataFrame consolidado:")
df_streaming.head(5)

In [ ]:
genre_mapping = {
    # --- PADRONIZAÇÃO E CONSOLIDAÇÃO (TV/Filmes -> Gênero Principal) ---
    'Dramas': 'Drama', 'TV Dramas': 'Drama',
    'Comedies': 'Comedy', 'TV Comedies': 'Comedy', 'Romantic Comedy': 'Romance , Comedy',
    'Thrillers': 'Thriller', 'TV Thrillers': 'Thriller',
    'Documentaries': 'Documentary', 'Docuseries': 'Documentary',
    'Horror Movies': 'Horror', 'TV Horror': 'Horror',
    'Romantic Movies': 'Romance', 'Romantic TV Shows': 'Romance',
    'International Movies': 'International', 'International TV Shows': 'International',
    'Independent Movies': 'Independent',
    'Sports Movies': 'Sports',
    'Classic Movies': 'Classic',
    'Cult Movies': 'Cult',
    'LGBTQ Movies': 'LGBTQ',
    'Crime TV Shows': 'Crime',
    'Reality TV': 'Reality',
    'Teen TV Shows': 'Teen',
    'TV Mysteries': 'Mystery',
    'Science Fiction': 'Sci-Fi',

    # --- MAPEAMENTO DE SINÔNIMOS E SUB-GÊNEROS ---
    'Anime Features': 'Anime', 'Anime Series': 'Anime',
    "Kids' TV": "Kids", 'Children & Family Movies': 'Kids, Family',
    'Faith and Spirituality': 'Faith & Spirituality',
    'Young Adult Audience': 'Young Adult',
    'Soap Opera / Melodrama': 'Soap Opera',
    'and Culture': 'Culture', 

    # --- SEPARAÇÃO DE GÊNEROS COMPOSTOS (usando vírgula) ---
    'Animals & Nature': 'Nature',
    'Science & Nature': 'Nature, Science', 
    'Arts & Culture': 'Culture, Art',
    'Action & Adventure': 'Action, Adventure',
    'Sci-Fi & Fantasy': 'Sci-Fi, Fantasy',
    'Stand-Up Comedy & Talk Shows': 'Stand-Up Comedy, Talk Show',
    'Music Videos and Concerts': 'Music',
    'Music & Musicals': 'Music, Musical',
    'Science & Nature TV': 'Science, Nature',
    'Animals & Nature': 'Animals, Nature',
    'TV Action & Adventure': 'Action, Adventure',
    'TV Sci-Fi & Fantasy': 'Sci-Fi, Fantasy',
    'Game Show / Competition': 'Game Show, Competition',
    'Action-Adventure': 'Action, Adventure',
    'Classic & Cult TV': 'Classic, Cult',
    'Talk Show and Variety': 'Talk Show, Variety',
    
    # --- Mapeamento direto de gêneros de TV para manter a distinção se desejado ---
    'Korean TV Shows': 'Korean TV',
    'British TV Shows': 'British TV',
    'Spanish-Language TV Shows': 'Spanish TV',

    # --- Remoção de Formatos (não são gêneros temáticos) ---
    'Movies': '_REMOVE_',
    'Series': '_REMOVE_',
    'TV Shows': '_REMOVE_', 
    'TV Show': '_REMOVE_',
    'Anthology': '_REMOVE_',
    'Unscripted': '_REMOVE_', # Categoria muito ampla, coberta por Reality
    'Special Interest': '_REMOVE_' # Categoria muito genérica
}

def process_genres(genre_string):
    """
    Função para aplicar o mapeamento de gênero em uma string 
    que pode conter múltiplos gêneros.
    """
    if pd.isna(genre_string):
        return '' # Retorna string vazia 

    processed_genres = set()
    
    # 1. Separa os gêneros da string original (ex: "TV Dramas, TV Mysteries")
    initial_genres = genre_string.split(',')

    for genre in initial_genres:
        # 2. Limpa o whitespace (ex: " TV Dramas" -> "TV Dramas")
        clean_genre = genre.strip()

        # 3. Aplica o mapping. 
        mapped_value = genre_mapping.get(clean_genre, clean_genre)

        # 4. Processa o valor mapeado
        if mapped_value == '_REMOVE_':
            # Não faz nada, simplesmente ignora o gênero
            continue
        elif ',' in mapped_value:
            # Separa, limpa e adiciona cada sub-gênero
            sub_genres = mapped_value.split(',')
            for sub in sub_genres:
                processed_genres.add(sub.strip())
        else:
            # É um valor único, não vazio e não _REMOVE_
            if mapped_value:
                processed_genres.add(mapped_value)
    
    # ordenados alfabeticamente para consistência.
    return ', '.join(sorted(list(processed_genres)))

df_streaming['genres_processed'] = df_streaming['listed_in'].apply(process_genres)

# 4. (Opcional) Mostra o resultado das 10 primeiras linhas
print("Processamento concluído. Exemplo do resultado:")
print(df_streaming[['listed_in', 'genres_processed']].head(10))

print(df_streaming.dtypes)